In [1]:
!pip install fuzzywuzzy

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np
import string

c:\users\varun jadia\appdata\local\programs\python\python38\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [4]:
#set up df and target sentence for string matching
triggers = pd.read_csv('weather app sample conversations - more explicit triggers.csv', names = ['Sentences'])
triggers.dropna(inplace=True)
target_sent = 'What is the weather'

In [5]:
#classify based on fuzzy string ratio
triggers['fuzzydist'] = triggers['Sentences'].apply(lambda x: fuzz.ratio(x, target_sent))
triggers['label'] = triggers['fuzzydist'].apply(lambda x: x > 50)

In [6]:
triggers

,Sentences,fuzzydist,label
0,What's the weather there?,77,True
1,I wonder what the weather there is,57,True
2,What's the weather going to be?,68,True
3,What's the weather going to be this weekend?,54,True
4,What will the weather be?,82,True
5,What will be the weather be?,77,True
6,"Weather forecast, please.",32,False
7,"Weather report, please.",33,False
8,I want to know the weather.,65,True
9,Please tell me whether it'll rain.,45,False


In [7]:
sum(triggers['label'])/len(triggers)

0.4594594594594595

In [8]:
#clean sentences first and pass into fuzzywuzzy
triggers['sent_cleaned'] = triggers['Sentences'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
triggers['label_cleaned'] = triggers['sent_cleaned'].apply(lambda x: fuzz.ratio(x, target_sent) > 50)

In [9]:
sum(triggers['label_cleaned'])/len(triggers)

0.4594594594594595

In [10]:
#test using multiple trigger sents
target_sents = ['what is the weather', 'will it rain today', 'is it hot today', 'what is todays forecast', 'is it cold outside']
def classify_sent(sent): 
    scores = [fuzz.ratio(sent, target) for target in target_sents]
    print(scores)
    return max(scores) > 50

triggers['label_mult'] = triggers['sent_cleaned'].apply(lambda x: classify_sent(x))
sum(triggers['label_mult'])/len(triggers)

[76, 20, 37, 39, 15]
[60, 23, 24, 35, 15]
[67, 26, 36, 35, 13]
[52, 23, 32, 31, 13]
[79, 43, 26, 43, 19]
[74, 36, 24, 40, 18]
[29, 20, 21, 48, 24]
[30, 26, 22, 32, 26]
[71, 32, 29, 29, 23]
[47, 40, 26, 15, 28]
[50, 17, 32, 38, 51]
[32, 32, 35, 29, 70]
[39, 65, 38, 31, 35]
[36, 14, 15, 34, 25]
[50, 41, 44, 41, 72]
[44, 40, 49, 40, 65]
[56, 21, 40, 28, 37]
[41, 37, 40, 37, 37]
[60, 29, 43, 48, 37]
[29, 48, 31, 31, 19]
[35, 36, 38, 30, 18]
[21, 11, 23, 23, 54]
[29, 54, 31, 34, 21]
[53, 38, 41, 25, 35]
[65, 36, 33, 52, 58]
[24, 50, 43, 36, 65]
[44, 40, 49, 40, 65]
[31, 68, 80, 37, 32]
[29, 43, 32, 38, 26]
[71, 27, 34, 37, 59]
[26, 67, 52, 40, 27]
[29, 39, 42, 74, 24]
[58, 32, 34, 24, 14]
[78, 20, 32, 40, 15]
[29, 37, 31, 27, 48]
[62, 60, 36, 42, 17]
[46, 18, 34, 33, 43]


0.7297297297297297

In [11]:
#test on negative examples
f = open('samples.txt', 'r')

In [12]:
labels_cont = [classify_sent(sent) for sent in f]    

[39, 20, 24, 24, 23]
[28, 15, 20, 20, 23]
[24, 22, 13, 19, 22]
[18, 16, 14, 23, 16]
[27, 28, 30, 29, 22]
[35, 23, 21, 27, 23]
[34, 31, 24, 29, 34]
[23, 21, 15, 31, 21]
[15, 27, 25, 11, 35]
[28, 31, 24, 22, 23]
[25, 22, 12, 24, 14]
[21, 23, 24, 26, 27]
[35, 21, 23, 23, 29]
[38, 12, 25, 25, 15]
[15, 31, 33, 25, 27]
[13, 26, 27, 27, 29]
[31, 21, 22, 26, 21]
[33, 31, 27, 31, 28]
[33, 28, 29, 26, 12]
[34, 24, 18, 28, 24]
[28, 18, 16, 17, 15]
[32, 27, 29, 24, 22]
[19, 15, 8, 13, 9]
[21, 17, 20, 25, 12]
[43, 16, 17, 29, 24]
[35, 18, 26, 28, 30]
[13, 27, 18, 15, 23]
[29, 37, 27, 20, 22]
[38, 21, 19, 27, 21]
[33, 34, 32, 31, 26]
[19, 16, 14, 27, 13]
[33, 15, 11, 30, 24]
[8, 28, 26, 29, 25]
[26, 24, 28, 31, 36]
[27, 24, 25, 19, 20]
[25, 14, 15, 24, 16]
[13, 25, 21, 25, 17]
[13, 17, 15, 17, 10]
[23, 10, 18, 24, 13]
[22, 16, 19, 17, 16]
[18, 28, 21, 26, 14]
[17, 20, 29, 28, 37]
[26, 16, 21, 21, 22]
[22, 15, 18, 17, 18]
[22, 28, 26, 41, 25]
[29, 15, 22, 31, 30]
[24, 19, 14, 32, 24]
[24, 18, 19, 17,

[22, 14, 14, 15, 16]
[33, 27, 25, 28, 23]
[29, 19, 31, 24, 19]
[6, 25, 20, 26, 17]
[18, 16, 22, 15, 16]
[14, 24, 27, 18, 19]
[31, 13, 17, 38, 6]
[23, 31, 21, 21, 24]
[29, 24, 19, 15, 18]
[28, 20, 17, 19, 24]
[22, 25, 20, 35, 16]
[28, 21, 14, 15, 16]
[28, 26, 24, 22, 23]
[24, 35, 30, 29, 28]
[24, 28, 33, 32, 31]
[21, 21, 22, 22, 21]
[11, 11, 11, 16, 25]
[31, 28, 30, 17, 22]
[17, 21, 22, 22, 19]
[17, 32, 31, 25, 24]
[27, 20, 13, 31, 12]
[31, 18, 19, 17, 21]
[31, 31, 26, 34, 20]
[30, 21, 19, 23, 30]
[18, 15, 16, 19, 27]
[16, 21, 22, 27, 21]
[14, 18, 17, 14, 20]
[12, 30, 25, 8, 24]
[25, 14, 14, 22, 14]
[24, 31, 21, 19, 13]
[19, 29, 21, 21, 19]
[33, 33, 18, 22, 20]
[34, 27, 19, 20, 24]
[29, 25, 27, 23, 33]
[19, 15, 20, 28, 27]
[23, 21, 20, 20, 21]
[43, 44, 37, 43, 39]
[32, 19, 19, 20, 19]
[39, 26, 21, 30, 23]
[35, 22, 29, 40, 22]
[31, 26, 25, 22, 24]
[25, 26, 14, 27, 16]
[26, 33, 24, 21, 13]
[21, 27, 14, 18, 22]
[28, 26, 16, 32, 23]
[19, 21, 25, 27, 21]
[29, 19, 22, 34, 29]
[23, 16, 19, 13,

[35, 26, 27, 36, 35]
[38, 26, 28, 47, 35]
[35, 35, 21, 21, 24]
[29, 24, 22, 27, 26]
[17, 25, 26, 30, 17]
[22, 29, 20, 30, 23]
[16, 13, 10, 20, 20]
[28, 21, 22, 24, 19]
[47, 30, 31, 37, 19]
[32, 24, 28, 30, 21]
[31, 31, 27, 32, 20]
[29, 26, 31, 14, 15]
[20, 20, 21, 29, 14]
[24, 27, 19, 22, 30]
[31, 17, 27, 37, 29]
[36, 36, 39, 33, 41]
[18, 8, 15, 8, 18]
[37, 35, 12, 24, 12]
[24, 21, 22, 19, 14]
[24, 18, 25, 25, 24]
[23, 18, 19, 19, 22]
[21, 30, 33, 31, 22]
[20, 20, 22, 13, 25]
[26, 15, 16, 19, 18]
[26, 22, 20, 29, 18]
[17, 21, 22, 24, 23]
[26, 19, 17, 25, 16]
[21, 21, 23, 35, 26]
[17, 16, 10, 21, 13]
[19, 30, 20, 28, 11]
[27, 18, 19, 26, 14]
[24, 27, 28, 28, 27]
[22, 16, 19, 15, 17]
[34, 31, 29, 29, 24]
[22, 20, 18, 30, 17]
[34, 26, 21, 32, 23]
[20, 12, 9, 34, 29]
[22, 22, 16, 31, 22]
[39, 23, 31, 27, 33]
[22, 19, 20, 21, 25]
[21, 18, 14, 17, 24]
[35, 20, 24, 24, 23]
[47, 30, 33, 39, 35]
[15, 22, 20, 32, 28]
[19, 49, 37, 22, 29]
[17, 39, 41, 39, 18]
[29, 18, 12, 31, 21]
[32, 43, 59, 14,

[40, 27, 25, 25, 20]
[26, 19, 20, 29, 19]
[28, 31, 30, 29, 17]
[15, 22, 16, 14, 28]
[25, 17, 26, 29, 25]
[30, 34, 29, 20, 25]
[21, 17, 20, 16, 19]
[24, 29, 9, 19, 29]
[36, 23, 21, 18, 20]
[18, 14, 15, 24, 12]
[18, 24, 21, 25, 20]
[25, 19, 24, 30, 13]
[32, 18, 31, 23, 33]
[39, 26, 24, 27, 16]
[20, 21, 18, 10, 24]
[24, 26, 14, 21, 20]
[23, 16, 27, 20, 24]
[18, 21, 20, 19, 13]
[24, 24, 19, 25, 27]
[20, 26, 18, 19, 22]
[28, 29, 36, 30, 33]
[30, 30, 16, 31, 15]
[16, 12, 18, 21, 11]
[23, 14, 21, 30, 5]
[31, 23, 24, 30, 23]
[20, 21, 17, 14, 41]
[26, 19, 11, 25, 29]
[30, 18, 11, 25, 22]
[28, 23, 21, 19, 15]
[34, 23, 30, 32, 26]
[33, 38, 36, 17, 19]
[37, 17, 15, 32, 20]
[21, 15, 15, 10, 15]
[18, 21, 19, 18, 21]
[15, 14, 19, 23, 22]
[24, 29, 26, 26, 20]
[14, 21, 22, 22, 32]
[32, 16, 9, 15, 12]
[15, 15, 18, 12, 15]
[15, 29, 22, 20, 31]
[30, 24, 19, 20, 18]
[18, 17, 13, 16, 13]
[21, 24, 18, 12, 22]
[8, 20, 18, 33, 25]
[12, 16, 16, 18, 18]
[32, 25, 26, 33, 20]
[27, 21, 22, 25, 17]
[31, 27, 33, 25, 

[24, 24, 28, 30, 22]
[27, 16, 15, 24, 16]
[30, 28, 23, 16, 32]
[35, 29, 20, 15, 23]
[16, 15, 18, 11, 18]
[23, 25, 18, 22, 20]
[26, 20, 13, 27, 26]
[15, 15, 18, 16, 22]
[9, 27, 34, 16, 27]
[24, 34, 32, 26, 24]
[38, 29, 32, 39, 49]
[25, 15, 22, 24, 23]
[15, 23, 19, 12, 18]
[39, 23, 19, 25, 23]
[22, 12, 19, 20, 19]
[19, 11, 8, 17, 9]
[21, 21, 13, 31, 24]
[21, 12, 15, 18, 19]
[32, 23, 24, 27, 26]
[28, 18, 22, 29, 14]
[22, 30, 36, 24, 15]
[22, 15, 15, 21, 18]
[14, 20, 21, 16, 23]
[21, 23, 17, 16, 16]
[27, 22, 23, 31, 25]
[17, 13, 14, 17, 19]
[32, 15, 13, 29, 15]
[13, 11, 10, 18, 13]
[25, 18, 24, 26, 15]
[32, 25, 12, 20, 28]
[23, 39, 25, 25, 13]
[22, 29, 21, 32, 20]
[25, 23, 21, 24, 14]
[19, 19, 19, 18, 11]
[29, 26, 35, 34, 33]
[19, 22, 22, 16, 19]
[8, 24, 21, 15, 24]
[22, 14, 10, 25, 16]
[31, 11, 19, 26, 20]
[21, 13, 14, 19, 15]
[28, 17, 18, 13, 23]
[12, 20, 21, 29, 24]
[15, 17, 11, 23, 6]
[29, 20, 18, 12, 15]
[18, 11, 19, 17, 23]
[28, 26, 24, 22, 23]
[49, 25, 38, 36, 30]
[25, 23, 20, 24, 1

...

[60, 26, 28, 43, 39]
[14, 18, 14, 17, 14]
[10, 17, 20, 20, 16]
[14, 31, 25, 25, 21]
[33, 21, 25, 28, 24]
[21, 32, 18, 19, 22]
[21, 18, 25, 27, 29]
[34, 23, 29, 21, 31]
[23, 19, 16, 27, 15]
[33, 37, 24, 37, 25]
[29, 18, 18, 22, 29]
[25, 21, 15, 16, 21]
[30, 22, 18, 28, 20]
[20, 17, 15, 21, 17]
[19, 20, 15, 11, 20]
[34, 17, 22, 29, 28]
[31, 24, 29, 35, 21]
[40, 16, 35, 30, 16]
[26, 20, 23, 25, 18]
[28, 20, 21, 19, 28]
[19, 25, 17, 21, 25]
[26, 24, 23, 30, 10]
[28, 22, 27, 24, 13]
[25, 39, 42, 26, 25]
[27, 12, 23, 33, 18]
[19, 15, 13, 18, 13]
[15, 16, 20, 16, 21]
[15, 13, 17, 23, 9]
[29, 11, 14, 30, 24]
[39, 23, 28, 31, 20]
[22, 18, 19, 24, 14]
[23, 19, 13, 14, 20]
[26, 29, 28, 32, 24]
[23, 23, 25, 18, 27]
[31, 25, 20, 35, 16]
[41, 31, 25, 44, 10]
[30, 19, 26, 31, 22]
[25, 23, 22, 24, 21]
[19, 13, 14, 18, 13]
[18, 10, 12, 21, 7]
[29, 18, 15, 30, 18]
[5, 22, 21, 21, 22]
[29, 22, 21, 21, 24]
[28, 26, 19, 24, 18]
[25, 17, 15, 27, 14]
[41, 38, 31, 26, 33]
[27, 17, 20, 16, 19]
[20, 18, 15, 21,

[25, 25, 26, 18, 15]
[35, 26, 33, 32, 36]
[34, 23, 27, 30, 23]
[24, 27, 19, 37, 24]
[22, 16, 14, 26, 22]
[17, 17, 13, 21, 29]
[30, 40, 24, 28, 22]
[25, 15, 10, 16, 15]
[24, 16, 19, 24, 23]
[24, 19, 16, 20, 14]
[33, 24, 25, 22, 14]
[21, 39, 25, 42, 42]
[27, 20, 30, 18, 36]
[34, 31, 36, 32, 28]
[23, 26, 24, 27, 20]
[31, 32, 33, 23, 21]
[17, 23, 21, 16, 21]
[33, 22, 23, 23, 25]
[23, 24, 19, 25, 24]
[14, 16, 19, 24, 16]
[29, 27, 20, 25, 14]
[28, 33, 31, 34, 24]
[23, 17, 20, 13, 15]
[32, 23, 25, 17, 23]
[31, 19, 33, 29, 25]
[18, 21, 29, 11, 24]
[19, 29, 20, 18, 14]
[26, 27, 12, 25, 31]
[25, 23, 24, 29, 23]
[29, 19, 22, 18, 24]
[22, 20, 13, 23, 20]
[21, 17, 16, 22, 19]
[27, 43, 29, 14, 24]
[20, 24, 26, 40, 24]
[12, 13, 12, 14, 13]
[20, 24, 26, 22, 16]
[37, 19, 20, 45, 23]
[15, 10, 9, 16, 15]
[24, 8, 8, 21, 22]
[23, 19, 18, 20, 13]
[28, 25, 18, 21, 23]
[21, 16, 23, 25, 22]
[16, 17, 29, 26, 14]
[24, 18, 12, 25, 24]
[12, 22, 12, 25, 22]
[32, 10, 15, 21, 17]
[32, 23, 21, 20, 23]
[25, 15, 16, 14,

[31, 24, 21, 34, 20]
[23, 21, 25, 20, 32]
[31, 14, 18, 21, 20]
[35, 24, 20, 25, 16]
[32, 12, 21, 24, 20]
[13, 27, 25, 34, 27]
[23, 11, 17, 14, 19]
[33, 17, 25, 22, 37]
[16, 24, 17, 25, 24]
[21, 22, 29, 23, 37]
[16, 28, 22, 20, 26]
[21, 25, 15, 13, 15]
[26, 21, 25, 22, 19]
[34, 19, 24, 21, 23]
[35, 23, 24, 33, 36]
[43, 36, 29, 36, 27]
[30, 17, 16, 14, 20]
[37, 21, 22, 11, 29]
[32, 21, 18, 25, 26]
[27, 27, 23, 25, 22]
[23, 29, 22, 27, 32]
[16, 29, 13, 22, 16]
[33, 21, 22, 29, 18]
[33, 24, 25, 28, 24]
[41, 26, 17, 36, 26]
[24, 22, 21, 19, 18]
[8, 14, 15, 27, 12]
[29, 15, 15, 28, 17]
[23, 28, 20, 21, 19]
[24, 31, 26, 26, 28]
[38, 13, 14, 38, 30]
[20, 16, 19, 22, 12]
[31, 26, 29, 28, 37]
[34, 28, 16, 20, 9]
[21, 16, 25, 25, 21]
[34, 23, 21, 30, 20]
[27, 27, 23, 26, 20]
[28, 23, 31, 15, 34]
[35, 23, 29, 31, 13]
[11, 21, 14, 11, 15]
[39, 25, 24, 16, 25]
[18, 16, 15, 19, 19]
[29, 9, 21, 22, 20]
[9, 18, 22, 39, 18]
[22, 22, 21, 26, 20]
[32, 23, 17, 39, 20]
[34, 23, 21, 27, 32]
[32, 25, 23, 30, 

[25, 21, 22, 30, 23]
[19, 20, 18, 22, 18]
[26, 15, 13, 27, 7]
[20, 30, 27, 21, 24]
[31, 16, 20, 26, 16]
[29, 19, 18, 23, 19]
[24, 22, 26, 23, 27]
[32, 21, 25, 25, 21]
[29, 25, 16, 22, 25]
[14, 29, 18, 32, 17]
[8, 20, 24, 26, 17]
[19, 29, 30, 24, 22]
[32, 22, 23, 26, 30]
[34, 16, 23, 29, 20]
[28, 17, 22, 19, 28]
[26, 17, 12, 16, 15]
[22, 25, 21, 32, 23]
[32, 18, 23, 23, 25]
[31, 29, 30, 24, 32]
[24, 19, 19, 11, 17]
[19, 16, 20, 29, 32]
[27, 27, 34, 37, 32]
[25, 25, 29, 18, 31]
[44, 20, 22, 31, 30]
[14, 11, 10, 13, 3]
[13, 32, 25, 27, 21]
[25, 22, 20, 12, 25]
[32, 19, 17, 33, 25]
[29, 13, 14, 30, 23]
[19, 17, 12, 22, 15]
[20, 22, 19, 16, 20]
[38, 21, 26, 29, 21]
[23, 25, 22, 18, 25]
[15, 22, 20, 14, 22]
[16, 23, 21, 28, 18]
[19, 29, 41, 21, 14]
[23, 31, 33, 13, 26]
[26, 22, 24, 18, 11]
[16, 8, 9, 15, 6]
[14, 22, 9, 17, 22]
[29, 17, 15, 21, 27]
[31, 17, 20, 29, 21]
[40, 22, 18, 14, 18]
[28, 18, 33, 29, 39]
[23, 24, 22, 19, 35]
[5, 21, 11, 17, 14]
[24, 32, 21, 22, 28]
[29, 29, 28, 21, 22]


[26, 15, 9, 27, 20]
[36, 15, 16, 31, 19]
[16, 27, 23, 31, 25]
[22, 22, 17, 31, 25]
[16, 21, 14, 24, 23]
[19, 25, 20, 13, 25]
[13, 23, 19, 27, 28]
[23, 35, 38, 21, 39]
[23, 25, 15, 20, 15]
[20, 23, 21, 22, 25]
[29, 13, 19, 30, 21]
[20, 24, 21, 29, 20]
[28, 25, 33, 24, 25]
[6, 31, 26, 24, 17]
[15, 23, 16, 17, 34]
[33, 18, 25, 25, 30]
[19, 23, 24, 38, 19]
[30, 15, 24, 17, 30]
[25, 23, 17, 28, 12]
[29, 20, 32, 35, 29]
[7, 15, 18, 28, 17]
[24, 18, 10, 19, 12]
[30, 15, 18, 19, 15]
[33, 25, 29, 37, 34]
[17, 19, 20, 22, 17]
[23, 21, 20, 27, 21]
[26, 18, 18, 17, 25]
[39, 33, 31, 39, 26]
[19, 27, 24, 18, 31]
[34, 32, 24, 27, 23]
[32, 21, 25, 27, 24]
[20, 18, 21, 16, 25]
[31, 23, 24, 32, 17]
[26, 20, 14, 23, 22]
[23, 19, 16, 27, 19]
[22, 44, 29, 16, 31]
[41, 38, 32, 21, 30]
[22, 19, 19, 27, 19]
[25, 11, 17, 24, 9]
[25, 23, 26, 29, 30]
[16, 29, 22, 15, 19]
[24, 29, 17, 35, 24]
[27, 16, 21, 25, 20]
[17, 23, 9, 24, 28]
[23, 13, 15, 24, 15]
[34, 17, 20, 23, 22]
[26, 30, 28, 35, 26]
[27, 21, 15, 25, 1

In [13]:
sum(labels_cont)

25

In [14]:
classify_sent("Im headed to New York this summer since my internship is remote anyway. Oh, that sounds cool! I've heard NY gets really humid at that time of the year though? I don't mind a little humidity, I think it'll be fine! Sweet! Let me know how that goes - I might have to go to Florida for mine, so well be on the same time zone. Oh, awesome")

[9, 4, 4, 10, 9]


False

## Testing framework

In [15]:
import test

In [20]:
def is_trigger(sent, trig): 
    return fuzz.ratio(sent, trig) > 70 

test.compute_train_test_split(is_trigger)

 16%|█████████████▎                                                                     | 8/50 [00:00<00:00, 68.25it/s]

Computing false positives:


100%|██████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<00:00, 77.95it/s]

(0, 0.0)
Compute true positives
(8, 0.5714285714285714)
